In [194]:
import pandas as pd

In [5]:
# Add columns from seed person and houeshold data to final output

In [6]:
df_syn_pop = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\adjusted_synthetic_persons.csv')

In [7]:
df_syn_hh = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\adjusted_synthetic_households.csv')

In [8]:
df_seed_pop = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\populationsim_inputs\data\seed_persons.csv')

In [9]:
df_seed_hh = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\populationsim_inputs\data\seed_households.csv')

In [10]:
df_syn_pop.head()

,PUMA,block_group_id,hh_id,household_id,AGEP,SEX,SCH,is_worker,per_num,NP,person_id,clone,new_per_num
0,11802,5335919001,63683,1,27,1,1.0,1,1,1,1,0,NaN
1,11802,5335919001,62898,2,33,1,1.0,1,1,1,2,0,NaN
2,11802,5335919001,38731,3,73,2,1.0,0,1,1,3,0,NaN
3,11802,5335919001,8704,4,65,2,1.0,0,1,1,4,0,NaN
4,11802,5335919001,2637,5,67,1,1.0,1,1,1,5,0,NaN


In [11]:
df_seed_pop.head()

,SERIALNO,RT,SERIALNO.1,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,...,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80,hhnum,is_worker,non_worker
0,2013000000006,P,2013000000006,9,1,11606,4,53,1061971,27,...,24,7,27,8,46,25,50,1,0,1
1,2013000000006,P,2013000000006,9,2,11606,4,53,1061971,22,...,20,7,25,8,41,22,47,1,0,1
2,2013000000038,P,2013000000038,9,1,11505,4,53,1061971,4,...,8,8,1,4,6,1,4,2,1,0
3,2013000000038,P,2013000000038,9,2,11505,4,53,1061971,4,...,8,7,1,4,7,2,4,2,1,0
4,2013000000038,P,2013000000038,9,3,11505,4,53,1061971,7,...,14,12,2,8,13,3,6,2,0,1


In [12]:
# Household Records
df_new_pop = df_syn_pop.merge(df_seed_pop[['ESR','PINCP','SCHL','SCHG','WKHP','RELP','hhnum','SPORDER']],
                left_on=['hh_id','per_num'], right_on=['hhnum','SPORDER'])
df_new_pop.columns = [i.lower() for i in df_new_pop.columns]
df_new_pop.drop(['new_per_num'], axis=1, inplace=True)

In [13]:
len(df_new_pop)

4048896

In [14]:
# Load adjusted household seeds
df_seed_hh_adj = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\adjusted_seed_households.csv')

In [15]:
# Household Records
df_new_hh = df_syn_hh.merge(df_seed_hh_adj[['hhnum','TEN','PUMA','NP']], left_on=['hh_id','PUMA','NP'], 
                            right_on=['hhnum','PUMA','NP'], how='left')

# Drop the NP column and replace with NP_adjusted - this represents the persons added to this household
df_new_hh['NP'] = df_new_hh['NP_adjusted'].copy()
df_new_hh.drop(['NP_adjusted','NP_adjusted','hh_id','new_pums_hh_id'], axis=1, inplace=True)

In [16]:
print(len(df_new_hh))
print(len(df_syn_hh))

1605264
1605264


In [126]:
df_new_hh.head()

,household_id,PUMA,block_group_id,NP,VEH,worker_count,HHT,HINCP,WGTP,hhnum,TEN
0,1,11802,5335919001,1,2.0,1,4.0,85000.0,30,63683,3.0
1,2,11802,5335919001,1,2.0,1,4.0,50000.0,45,62898,3.0
2,3,11802,5335919001,1,3.0,0,6.0,56000.0,64,38731,2.0
3,4,11802,5335919001,1,3.0,0,6.0,51000.0,42,8704,2.0
4,5,11802,5335919001,1,3.0,1,4.0,74050.0,24,2637,1.0


In [17]:
# df_new_hh.to_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_households.csv',
#                  index=False)

In [18]:
# df_new_pop.to_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_persons.csv',
#                  index=False)

In [19]:
# Update block group ID

In [20]:
block_group_table = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\census_block_groups.csv')

In [21]:
# 2+3+6+1=12
# state county tract block group

In [22]:
# Drop leading zeros to match format of synthetic population
block_group_table['state'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[0:2]))
block_group_table['county'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[3:5]))
block_group_table['tract'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[5:11]))
block_group_table['block_group'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[11:]))

block_group_table['newid'] = block_group_table['state'].astype('str') + block_group_table['county'].astype('str') + block_group_table['tract'].astype('str') + block_group_table['block_group'].astype('str')
# block_group_table['newid'] = block_group_table['newid'].astype('int')

In [23]:
df_new_hh['newid'] = df_new_hh['block_group_id'].astype('str')

In [24]:
df = block_group_table.merge(df_new_hh, on='newid')

In [25]:
df.head()

,census_2010_block_group_id,census_block_group_id,census_tract_id,state,county,tract,block_group,newid,household_id,PUMA,block_group_id,NP,VEH,worker_count,HHT,HINCP,WGTP,hhnum,TEN
0,530330001001,1137,1,53,33,100,1,53331001,113026,11602,53331001,1,1.0,0,6.0,380700.0,14,51082,2.0
1,530330001001,1137,1,53,33,100,1,53331001,113027,11602,53331001,1,1.0,0,6.0,228800.0,21,1966,2.0
2,530330001001,1137,1,53,33,100,1,53331001,113028,11602,53331001,1,1.0,0,6.0,186000.0,14,73897,3.0
3,530330001001,1137,1,53,33,100,1,53331001,113029,11602,53331001,1,1.0,0,6.0,228800.0,21,1966,2.0
4,530330001001,1137,1,53,33,100,1,53331001,113030,11602,53331001,1,1.0,0,6.0,228800.0,21,1966,2.0


In [26]:
len(df_new_hh)

1605264

In [27]:
df.to_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_households.csv')

In [28]:
df_new_pop['newid'] = df_new_pop['block_group_id'].astype('str')
df_person = block_group_table.merge(df_new_pop, on='newid')

In [29]:
df_person.to_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_persons.csv',
                index=False)

In [30]:
test = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_persons.csv')

In [33]:
len(df_person)

4048896

In [34]:
len(df_new_pop)

4048896